In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import pandas as pd
import requests

# Load Delhi places data
delhi_places = pd.read_csv("delhi_places.csv")
place_names = delhi_places["Place Name"].tolist()

# FastAPI endpoints
DELIVERY_TIME_API = "http://127.0.0.1:8000/predict_delivery_time/"
ROUTE_OPTIMIZATION_API = "http://127.0.0.1:8000/optimize_routes/"

def get_lat_lon(place_name):
    """Get latitude and longitude for a given place name."""
    place = delhi_places[delhi_places["Place Name"] == place_name]
    return place.iloc[0]["Latitude"], place.iloc[0]["Longitude"]

def predict_delivery_time():
    """Fetch delivery time prediction from the FastAPI."""
    pickup = pickup_var.get()
    destination = destination_var.get()
    weight = weight_var.get()

    if not all([pickup, destination, weight]):
        messagebox.showerror("Input Error", "Please fill all fields.")
        return

    try:
        weight = float(weight)
    except ValueError:
        messagebox.showerror("Input Error", "Weight must be a numeric value.")
        return

    # Prepare API request payload
    payload = {
        "pickup_location": pickup,
        "delivery_location": destination,
        "order_weight": weight
    }

    try:
        response = requests.post(DELIVERY_TIME_API, json=payload)
        response_data = response.json()

        if "detail" in response_data:
            messagebox.showerror("API Error", response_data["detail"])
        else:
            result = f"Predicted Delivery Time: {response_data['predicted_delivery_time']} mins\n"
            result += f"Distance: {response_data['distance_km']} km\n"
            result += f"Traffic Duration: {response_data['traffic_duration_min']} mins\n"
            result += f"Weather: {response_data['weather_condition']} ({response_data['temperature_c']} °C)"
            messagebox.showinfo("Delivery Prediction", result)
    except requests.RequestException as e:
        messagebox.showerror("Error", f"Failed to connect to API. {str(e)}")
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {str(e)}")

def optimize_routes():
    """Fetch optimized route from the FastAPI."""
    pickup = pickup_var.get()
    destinations = destinations_text.get("1.0", tk.END).strip().split("\n")

    if not all([pickup, destinations]):
        messagebox.showerror("Input Error", "Please fill all fields.")
        return

    # Prepare API request payload
    payload = {
        "pickup_location": pickup,
        "delivery_locations": destinations
    }

    try:
        response = requests.post(ROUTE_OPTIMIZATION_API, json=payload)
        response_data = response.json()

        if "detail" in response_data:
            messagebox.showerror("API Error", response_data["detail"])
        else:
            route = " -> ".join(response_data['optimized_route'])
            result = f"Optimized Route:\n{route}\n\nTotal Distance: {response_data['total_distance_km']} km"
            messagebox.showinfo("Route Optimization", result)
    except requests.RequestException as e:
        messagebox.showerror("Error", f"Failed to connect to API. {str(e)}")
    except Exception as e:
        messagebox.showerror("Error", f"An error occurred: {str(e)}")

# Create main Tkinter window
root = tk.Tk()
root.title("Delivery Route Optimization")
root.geometry("500x600")

# Pickup location dropdown
pickup_label = tk.Label(root, text="Pickup Location:")
pickup_label.pack(pady=5)
pickup_var = tk.StringVar()
pickup_dropdown = ttk.Combobox(root, textvariable=pickup_var, values=place_names)
pickup_dropdown.pack(pady=5)

# Delivery location dropdown
destination_label = tk.Label(root, text="Delivery Location:")
destination_label.pack(pady=5)
destination_var = tk.StringVar()
destination_dropdown = ttk.Combobox(root, textvariable=destination_var, values=place_names)
destination_dropdown.pack(pady=5)

# Order weight input
weight_label = tk.Label(root, text="Order Weight (kg):")
weight_label.pack(pady=5)
weight_var = tk.StringVar()
weight_entry = tk.Entry(root, textvariable=weight_var)
weight_entry.pack(pady=5)

# Button for delivery time prediction
predict_button = tk.Button(root, text="Predict Delivery Time", command=predict_delivery_time)
predict_button.pack(pady=10)

# Delivery locations for route optimization
destinations_label = tk.Label(root, text="Delivery Locations (one per line):")
destinations_label.pack(pady=5)
destinations_text = tk.Text(root, height=10, width=50)
destinations_text.pack(pady=5)

# Button for route optimization
optimize_button = tk.Button(root, text="Optimize Routes", command=optimize_routes)
optimize_button.pack(pady=10)

# Run the Tkinter main loop
root.mainloop()